In [6]:
import warnings
warnings.filterwarnings('ignore')

import importlib.util
from sklearn.linear_model import Ridge, RidgeCV, BayesianRidge
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

from sklearn.model_selection import cross_val_score, RandomizedSearchCV, RepeatedKFold, GridSearchCV
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

import xgboost as xg

import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict

import altair as alt
from altair_saver import save
from scipy.stats import skew
import joblib
import json

alt.renderers.enable('default'); # if in jupyter, ; to suppress output

# Regression Exploration

In this notebook we will explore several types of regression models to see which type seems to fit the data best. We will dive deeper into a few of the best performing models to see if there is any way to improve them.

--------
## Data Cleaning

### Set Up

Time columns that need to be converted to integer hour for model:

In [2]:
parse_times = ["MKOPEN", "MKCLOSE", "MKEMHOPEN", "MKEMHCLOSE",
               "MKOPENYEST", "MKCLOSEYEST", "MKOPENTOM",
               "MKCLOSETOM","EPOPEN", "EPCLOSE", "EPEMHOPEN",
               "EPEMHCLOSE", "EPOPENYEST", "EPCLOSEYEST",
               "EPOPENTOM", "EPCLOSETOM", "HSOPEN", "HSCLOSE",
               "HSEMHOPEN", "HSEMHCLOSE", "HSOPENYEST", "HSCLOSEYEST",
               "HSOPENTOM", "HSCLOSETOM", "AKOPEN", "AKCLOSE",
               "AKEMHOPEN", "AKOPENYEST", "AKCLOSEYEST","AKEMHCLOSE",
               "AKOPENTOM", "AKCLOSETOM", "MKPRDDT1", "MKPRDDT2",
               "MKPRDNT1", "MKPRDNT2", "MKFIRET1", "MKFIRET2",
               "EPFIRET1", "EPFIRET2", "HSPRDDT1", "HSFIRET1",
               "HSFIRET2", "HSSHWNT1", "HSSHWNT2", "AKPRDDT1",
               "AKPRDDT2", "AKSHWNT1", "AKSHWNT2"]

### Load posted wait time datasets

In [8]:
def setup():
    with open("../data/processed/dtypes_parsed.json") as json_file:
        dtypes = json.load(json_file)

    return dtypes


def loadTrainTestPostedWaitTimes():
    """
            Loads train test data for posted wait times

            How to use:

            import importlib.util

            spec = importlib.util.spec_from_file_location("loadTrainTestPostedWaitTimes", "src/data/loadTrainTestData.py")
            loadTrainPosted = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(loadTrainPosted)

            X_train_posted, X_test_posted, y_train_posted, y_test_posted = loadTrainPosted.loadTrainTestPostedWaitTimes()

            Parameters
            ----------

            Returns
            -------
            rideDataDf_trainX - train data features for posted wait times
            rideDataDf_testX - test data features for posted wait times
            rideDataDf_trainY - train data targets for posted wait times
            rideDataDf_testY - test data targets for posted wait times

        """
    parse_dates = ['date', 'datetime']
    X_train_list = []
    y_train_list = []
    dtypes = setup()

    for year in range(2015, 2022):

        rideData = pd.read_csv(f'../data/processed/All_train_postedtimes{year}.csv', dtype=dtypes,
                               parse_dates=parse_dates, compression='gzip')
        rideDataX = rideData.drop(columns=["POSTED_WAIT"])
        rideDataY = rideData["POSTED_WAIT"]
        X_train_list.append(rideDataX)
        y_train_list.append(rideDataY)

    rideDataDf_trainX = pd.concat(X_train_list, ignore_index=True)
    rideDataDf_trainY = pd.concat(y_train_list, ignore_index=True)


    X_test_list = []
    y_test_list = []

    for year in range(2015, 2022):
        rideData = pd.read_csv(f'../data/processed/All_test_postedtimes{year}.csv', dtype=dtypes,
                               parse_dates=parse_dates, compression='gzip')
        rideDataX = rideData.drop(columns=["POSTED_WAIT"])
        rideDataY = rideData["POSTED_WAIT"]

        X_test_list.append(rideDataX)
        y_test_list.append(rideDataY)

    rideDataDf_testX = pd.concat(X_test_list, ignore_index=True)
    rideDataDf_testY = pd.concat(y_test_list, ignore_index=True)

    return rideDataDf_trainX, rideDataDf_testX, rideDataDf_trainY, rideDataDf_testY

In [9]:
X_train, X_test, y_train, y_test = loadTrainTestPostedWaitTimes()

### Convert key data points from date to integer

In [10]:
X_train["MONTHOFYEAR"] = X_train["date"].dt.month.astype("Int8")
X_train["YEAR"] = X_train["date"].dt.year.astype("Int16")
X_train["DAYOFYEAR"] = X_train["date"].dt.dayofyear.astype("Int16")
X_train["HOUROFDAY"] = X_train["datetime"].dt.hour.astype("Int8")

X_test["MONTHOFYEAR"] = X_test["date"].dt.month.astype("Int8")
X_test["YEAR"] = X_test["date"].dt.year.astype("Int16")
X_test["DAYOFYEAR"] = X_test["date"].dt.dayofyear.astype("Int16")
X_test["HOUROFDAY"] = X_test["datetime"].dt.hour.astype("Int8")

### Sort by datetime before imputation (keeping y-values associated)

In [11]:
train = pd.concat([X_train, y_train], axis=1).sort_values(['datetime'])
test = pd.concat([X_test, y_test], axis=1).sort_values(['datetime'])

In [12]:
X_train_impute = train.drop(columns=["POSTED_WAIT"])
y_train = train["POSTED_WAIT"]

X_test_impute = test.drop(columns=["POSTED_WAIT"])
y_test = test["POSTED_WAIT"]

### Many open/close times, parade times, etc. are in HH:MM format. 

Convert to integer hour & fill nulls with 99.

This means that particulate event does not exist for that day. (e.g. Magic Kingdom doesn't have a second parade)

In [13]:
for col in parse_times:
    X_train_impute[col] =  X_train_impute[col].fillna("99")
    X_train_impute[f"{col}_HOUR"] = X_train_impute[col].apply(lambda x: x[:2] if x[0]!=0 else x[:1]).astype(int).astype("Int8")
    X_train_impute.drop(columns = col, inplace=True)

In [14]:
for col in parse_times:
    X_test_impute[col] =  X_test_impute[col].fillna("99")
    X_test_impute[f"{col}_HOUR"] = X_test_impute[col].apply(lambda x: x[:2] if x[0]!=0 else x[:1]).astype(int).astype("Int8")
    X_test_impute.drop(columns = col, inplace=True)

### Data Imputation

First impute by backfilling values. For any remaining nulls, impute the median.

In [15]:
for col in X_train_impute.columns:
    nulls = X_train_impute[col].isnull().sum()
    
    if nulls>0:
        X_train_impute[col].fillna(method ='bfill', inplace=True)
    
        if X_train_impute[col].isnull().sum()>0:
            X_train_impute[col].fillna(X_train_impute[col].median(), inplace=True)

In [16]:
for col in X_test_impute.columns:
    nulls = X_test_impute[col].isnull().sum()
    if nulls>0:
        X_test_impute[col]= X_test_impute[col].fillna(method ='bfill')
        
        if X_test_impute[col].isnull().sum()>0:
            X_test_impute[col].fillna(X_test_impute[col].median(), inplace=True)

### Drop Datetime columns - no longer needed

In [17]:
X_train_encoded = X_train_impute.drop(columns=['date', 'datetime', 'Unnamed: 0'])
X_test_encoded = X_test_impute.drop(columns=['date', 'datetime', 'Unnamed: 0'])

In [18]:
del X_train_impute, X_test_impute

### DROP EVENTUALLY: Use variance threshold to drop more boolean columns.

These columns are more than 99.9% similar.

In [19]:
X_dtype = X_train_encoded.select_dtypes(include=['bool']).reset_index(drop=True)

var_thr = VarianceThreshold(threshold=0.001)  # Removing both constant and quasi-constant
var_thr.fit(X_dtype)

concol = [column for column in X_dtype.columns
          if column not in X_dtype.columns[var_thr.get_support()]]


del var_thr, X_dtype

if "Weather Type" in concol:
    concol.remove("Weather Type")

print(f"DROPPING BOOL: ", concol)
X_train_encoded.drop(concol, axis=1, inplace=True)
X_test_encoded.drop(concol, axis=1, inplace=True)

DROPPING BOOL:  []


### Data Scaling & Transformation

Log Transform skewed numeric columns & then apply StandardScaler to scale numeric columns.

In [20]:
scaler = RobustScaler()


X_dtype_train = X_train_encoded.select_dtypes(include=[np.number]).reset_index(drop=True)
num_cols = list(X_dtype_train.columns)

X_dtype_test = X_test_encoded.select_dtypes(include=[np.number]).reset_index(drop=True)

In [21]:
skewed_cols = []
for col in X_dtype_train.columns:
    skewness = abs(skew(list(X_dtype_train[col])))
    if skewness > 1:
        X_dtype_train[f"log_{col}"] = np.log(X_dtype_train[col].astype(float).apply(lambda x: x+5))
        X_dtype_test[f"log_{col}"] = np.log(X_dtype_test[col].astype(float).apply(lambda x: x+5))                                         
        skewed_cols.append(col)

X_dtype_train.drop(columns=skewed_cols, inplace=True)
X_dtype_test.drop(columns=skewed_cols, inplace=True)

In [22]:
X_train_norm = scaler.fit_transform(X_dtype_train)
X_test_norm = scaler.transform(X_dtype_test)

X_train_encoded[num_cols] = X_train_norm
X_test_encoded[num_cols] = X_test_norm

In [23]:
del X_train_norm, X_test_norm, X_dtype_train, X_dtype_test

------

# Modeling

## Dummy Regression - Baseline

Setting baseline metrics to see improvement among other models. These models are not expected to do well.

In [18]:
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train_encoded, y_train)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test_encoded)

In [19]:
print("DUMMY REGRESSOR - MEAN")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_predict_dummy_mean))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_predict_dummy_mean))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_predict_dummy_mean)))
print("R-SQUARED: ", metrics.r2_score(y_test, y_predict_dummy_mean))

DUMMY REGRESSOR - MEAN
Mean Absolute Error (MAE): 162.41068811170675
Mean Squared Error (MSE): 84453.25944579137
Root Mean Squared Error (RMSE): 290.6084297569349
R-SQUARED:  -3.6194275621070915e-07


In [20]:
lm_dummy_median = DummyRegressor(strategy = 'median').fit(X_train_encoded, y_train)
y_predict_dummy_median = lm_dummy_median.predict(X_test_encoded)

In [21]:
print("DUMMY REGRESSOR - MEDIAN")

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_predict_dummy_median))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_predict_dummy_median))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_predict_dummy_median)))
print("R-SQUARED: ", metrics.r2_score(y_test, y_predict_dummy_median))

DUMMY REGRESSOR - MEDIAN
Mean Absolute Error (MAE): 106.43553661159103
Mean Squared Error (MSE): 90904.36571096092
Root Mean Squared Error (RMSE): 301.5035086213109
R-SQUARED:  -0.07638709517762399


## Ridge Regression w/ Grid Search on Alpha

Testing ridge regression with differing levels of alpha. This model does not do very well compared to the dummy regressors. We will not move forward with ridge regression.

In [22]:
linRidge = RidgeCV(alphas=[1e-1, 1, 10], scoring='neg_mean_absolute_error').fit(X_train_encoded, y_train)
predLinRidge = linRidge.predict(X_test_encoded)

In [24]:
print("RIDGE REGRESSION GRID SEARCH: ")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predLinRidge))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predLinRidge))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predLinRidge)))
print("R-SQUARED: ", metrics.r2_score(y_test, predLinRidge))

RIDGE REGRESSION GRID SEARCH: 
Mean Absolute Error (MAE): 141.39446762707306
Mean Squared Error (MSE): 69836.10725051745
Root Mean Squared Error (RMSE): 264.2652214168892
R-SQUARED:  0.17307948816330976


## XGBoost Regressor

```n_estimators = 100```

XGBoost regressor does significantly better than the dummy regressors, however it's not doing as well as we'd hope. We will put this one aside for now and continue exploring.

In [25]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 100, seed = 123)
# Fitting the model
xgb_r.fit(X_train_encoded, y_train)
# Predict the model
pred = xgb_r.predict(X_test_encoded)

In [26]:
print("XGBoost n_estimators = 100")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print("R-SQUARED: ", metrics.r2_score(y_test, pred))

XGBoost n_estimators = 100
Mean Absolute Error (MAE): 81.98376024978742
Mean Squared Error (MSE): 38018.90601036953
Root Mean Squared Error (RMSE): 194.9843737594619
R-SQUARED:  0.5498229432406851


## Bayesian Ridge Regressor

Bayesian Ridge regressor has similar results to traditional Ridge which does not perform well. We will not move forward with this model.

In [27]:
BayReg = BayesianRidge()
BayReg.fit(X_train_encoded, y_train)
pred = BayReg.predict(X_test_encoded)

In [28]:
print("Bayesian Ridge Regression: ")
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print("R-SQUARED: ", metrics.r2_score(y_test, pred))

Bayesian Ridge Regression: 
Mean Absolute Error (MAE): 141.4040042726355
Mean Squared Error (MSE): 69840.6091746251
Root Mean Squared Error (RMSE): 264.2737390938137
R-SQUARED:  0.1730261814494347


-----------------------

# Tree-Based Model Deep Dive

Our analysis showed us that tree based models do fairly well on this data. We will further explore these models to determine if we can maximize the performance and minimize the complexity of these models.

## Random Forest Regression

We resorted to doing hyperparameter tuning manually because GCP threw a TerminatedWorkerError during GridSearchCV and RandomizedSearchCV efforts and running the same thing locally threw memory errors.

In [ ]:
important_features = {}
rfResults = {}
feature_dict = defaultdict(list)
rf_feature_importance = defaultdict(dict)
for n_est in (10, 50, 100):
    for max_depth in [10, 50, 100]:
        print(f"STARTING {n_est}, {max_depth}")
        rfc = RandomForestRegressor(n_estimators=n_est, max_depth=max_depth, random_state=0, n_jobs=-1)
        rfc.fit(X_train_encoded, y_train)
        
        pred = rfc.predict(X_test_encoded)
        
        mae = metrics.mean_absolute_error(y_test, pred)
        mse = metrics.mean_squared_error(y_test, pred)
        rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))
        r2 = metrics.r2_score(y_test, pred)
        
        rfResults[f"{n_est}_{max_depth}"] = {"mae": mae, "mse": mse, "rmse":rmse, "r2":r2}
        
        importances = rfc.feature_importances_
        important_feat = np.argsort(importances)[-50:]
        
        selected_feat_names= list(X_train.columns[important_feat])
        
        for idx, x in enumerate(important_feat):
            feature_dict[selected_feat_names[idx]].append(importances[x])
            
            try:
                rf_feature_importance[selected_feat_names[idx]]["count"] += 1
                rf_feature_importance[selected_feat_names[idx]]["importances"].append(importances[x])
            except KeyError:
                rf_feature_importance[selected_feat_names[idx]]["count"] = 1
                rf_feature_importance[selected_feat_names[idx]]["importances"] = [importances[x]]
            
    
print(rfResults)

STARTING 10, 10
STARTING 10, 50
STARTING 10, 100
STARTING 50, 10
STARTING 50, 50


## RandomForest Grid Search Results

It appears that the sweet spot for metrics optimization vs. model complexity is **10 estimators and max depth of 50 for each tree**. Additional trees beyond 10 does not add value to the model and there is a sigificant reduction in error and increase in r-squared after increasing max-depth to 50. However, more than 50 max_depth for each tree does not seem to increase value.

In [53]:
rfGridSearchResults = pd.DataFrame.from_dict(rfResults).T.reset_index()

rfGridSearchResults[['n_estimators', 'max_depth']] = rfGridSearchResults['index'].str.split("_", expand=True).astype(int)
rfGridSearchResults = rfGridSearchResults.melt(id_vars=["index", "n_estimators", "max_depth"])

rfMetricsChart = alt.Chart(rfGridSearchResults).mark_bar().encode(
    x='max_depth:N',
    y='value:Q',
    color=alt.Color('variable:N', scale = alt.Scale(range=["#EAC2B1", "#90C6FA", "#F8E467", "#2C7AAF"])),
    row='n_estimators:N',
    column='variable:N',
).resolve_scale(
  y='independent'
).properties(
    title={
    "text": ["Performance Metrics by n_estimators & max_depth"],
      "subtitle": ["Random Forest Model"]
    }
)

save(rfMetricsChart, "../reports/figures/GridSearchRandomForest.html")

rfMetricsChart

alt.Chart(...)

### Top Features for Random Forest

Top 50 Features sorted on how many of the 9 Random Forest models considered it important and mean importance among those models:

In [ ]:
rf_feature_importance

In [ ]:
top20=sorted(rf_feature_importance, key=lambda x: (-rf_feature_importance[x]['count'], -rf_feature_importance[x]["mean_importance"]))[:20]
rf_feature_importance[top20]

In [55]:
for x in rf_feature_importance:
    rf_feature_importance[x]["mean_importance"] = np.mean(rf_feature_importance[x]["importances"])
    
sorted(rf_feature_importance, key=lambda x: (-rf_feature_importance[x]['count'], -rf_feature_importance[x]["mean_importance"]))[:50]

['EPOPENYEST',
 'Age_interest_adults',
 'Age_interest_teens',
 'Height_req_inches',
 'WDWSEASON_none',
 'HSHOURSYEST',
 'EPHOURSYEST',
 'MKOPENTOM',
 'MKFIREN_fantasy in the sky fireworks',
 'AKOPEN',
 'EPEMHOPEN',
 'WDWSEASON_columbus day',
 'MKCLOSEYEST',
 'MKHOURSYEST',
 'MKCLOSETOM',
 'WDWSEASON_memorial day',
 'WDWSEASON_september low',
 'DAYOFYEAR',
 'MKOPENYEST',
 'HOLIDAYN_lab',
 'INSESSION_PLANES',
 'MKFIREN_happily ever after',
 'Age_of_ride_days',
 'MKeventN_mvmcp',
 'HSOPENTOM',
 'Age_of_ride_years',
 'MKeventN_mnsshp',
 'WDWSEASON_thanksgiving',
 'inSession_NY_NJ',
 'WDWeventN_wdwdd|wdwhol',
 'EPCLOSEYEST',
 'Wind Speed Quality_p',
 'Ride_type_slow',
 'MKPRDNN_main street electrical parade',
 'MKEMHCLOSE',
 'Ride_name_dumbo the flying elephant',
 'HSEMHEYEST',
 'inSession',
 'MKEMHMORN',
 'MKPRDNN_mickey\'s "boo-to-you" halloween parade',
 'MKPRDNN_none',
 "MKFIREN_disney's celebrate america! - a fourth of july concert in the sky",
 'HSHOURSTOM',
 'WEEKOFYEAR',
 "MKFIREN_d

## Extra Trees Regressor

Extra Trees regressor is supposed to help with overfitting in RandomForest models. We tried this model to see if we could improve the performance on the test set, however, we got very similar results to Random Forest so have decided not to go with this model.

In [ ]:
important_features = {}
etResults = {}
feature_dict = defaultdict(list)
et_feature_importance = defaultdict(dict)
for n_est in (10, 50, 100):
    for max_depth in [10, 50, 100]:
        print(f"STARTING {n_est}, {max_depth}")
        etr = ExtraTreesRegressor(n_estimators=n_est, max_depth=max_depth, random_state=0, n_jobs=-1)
        etr.fit(X_train_encoded, y_train)
        
        pred = etr.predict(X_test_encoded)
        
        mae = metrics.mean_absolute_error(y_test, pred)
        mse = metrics.mean_squared_error(y_test, pred)
        rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))
        r2 = metrics.r2_score(y_test, pred)
        
        etResults[f"{n_est}_{max_depth}"] = {"mae": mae, "mse": mse, "rmse":rmse, "r2":r2}
        
        importances = etr.feature_importances_
        important_feat = np.argsort(importances)[-50:]
        
        selected_feat_names= list(X_train.columns[important_feat])
        
        for idx, x in enumerate(important_feat):
            feature_dict[selected_feat_names[idx]].append(importances[x])
            
            try:
                et_feature_importance[selected_feat_names[idx]]["count"] += 1
                et_feature_importance[selected_feat_names[idx]]["importances"].append(importances[x])
            except KeyError:
                et_feature_importance[selected_feat_names[idx]]["count"] = 1
                et_feature_importance[selected_feat_names[idx]]["importances"] = [importances[x]]
            
    
print(etResults)

## Extra Trees Grid Search Results

It appears that the sweet spot for metrics optimization vs. model complexity is 10 estimators and max of 50 trees.

In [54]:
etGridSearchResults = pd.DataFrame.from_dict(etResults).T.reset_index()

etGridSearchResults[['n_estimators', 'max_depth']] = etGridSearchResults['index'].str.split("_", expand=True).astype(int)
etGridSearchResults = etGridSearchResults.melt(id_vars=["index", "n_estimators", "max_depth"])

etMetricsChart = alt.Chart(etGridSearchResults).mark_bar().encode(
    x='max_depth:N',
    y='value:Q',
    color=alt.Color('variable:N', scale = alt.Scale(range=["#EAC2B1", "#90C6FA", "#F8E467", "#2C7AAF"])),
    row='n_estimators:N',
    column='variable:N',
).resolve_scale(
  y='independent'
).properties(
    title={
    "text": ["Performance Metrics by n_estimators & max_depth"],
      "subtitle": ["Extra Trees Model"]
    }
)

save(etMetricsChart, '../reports/figures/GridSearchExtraTrees.html')
etMetricsChart

alt.Chart(...)

### Top Features for Extra Trees

Top 50 Features sorted on how many of the 9 Extra trees models considered it important and mean importance among those models:

In [56]:
for x in et_feature_importance:
    et_feature_importance[x]["mean_importance"] = np.mean(et_feature_importance[x]["importances"])
    
sorted(et_feature_importance, key=lambda x: (-et_feature_importance[x]['count'], -et_feature_importance[x]["mean_importance"]))[:50]

['WDW_TICKET_SEASON_none',
 'Cloud Quality Code_passed all quality control checks',
 'CapacityLostWGT_MK',
 'CapacityLost_HS',
 'HSCLOSE',
 'HSOPEN',
 'EPHOURSTOM',
 'EPCLOSETOM',
 'EPOPENTOM',
 'EPHOURSYEST',
 'EPCLOSEYEST',
 'EPOPENYEST',
 'EPEMHCLOSE',
 'HSHOURS',
 'EPEMHOPEN',
 'EPCLOSE',
 'EPOPEN',
 'MKHOURSTOM',
 'MKCLOSETOM',
 'MKOPENTOM',
 'MKHOURSYEST',
 'MKCLOSEYEST',
 'MKOPENYEST',
 'MKEMHCLOSE',
 'EPHOURS',
 'CapacityLost_AK',
 'HSEMHOPEN',
 'HSOPENYEST',
 'CapacityLost_EP',
 'CapacityLost_MK',
 'WEATHER_WDWLOW',
 'WEATHER_WDWHIGH',
 'AKHOURSTOM',
 'AKCLOSETOM',
 'AKOPENTOM',
 'AKHOURSYEST',
 'AKCLOSEYEST',
 'HSEMHCLOSE',
 'AKOPENYEST',
 'AKEMHOPEN',
 'AKHOURS',
 'AKCLOSE',
 'AKOPEN',
 'HSHOURSTOM',
 'HSCLOSETOM',
 'HSOPENTOM',
 'HSHOURSYEST',
 'HSCLOSEYEST',
 'AKEMHCLOSE',
 'Cloud Quality Code_passed all quality control checks, data originate from an ncei data source']

### Decision Tree Regressor

In [30]:
important_features = {}
dtResults = {}
feature_dict = defaultdict(list)
dt_feature_importance = defaultdict(dict)
for max_depth in [10, 50, 100, None]:
    print(f"STARTING Max Depth {max_depth}")
    dt = DecisionTreeRegressor(max_depth=max_depth, random_state=0)
    dt.fit(X_train_encoded, y_train)

    pred = dt.predict(X_test_encoded)

    mae = metrics.mean_absolute_error(y_test, pred)
    mse = metrics.mean_squared_error(y_test, pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))
    r2 = metrics.r2_score(y_test, pred)

    dtResults[f"MaxDepth{max_depth}"] = {"mae": mae, "mse": mse, "rmse":rmse, "r2":r2}

    importances = dt.feature_importances_
    important_feat = np.argsort(importances)[-50:]

    selected_feat_names= list(X_train.columns[important_feat])

    for idx, x in enumerate(important_feat):
        feature_dict[selected_feat_names[idx]].append(importances[x])

        try:
            dt_feature_importance[selected_feat_names[idx]]["count"] += 1
            dt_feature_importance[selected_feat_names[idx]]["importances"].append(importances[x])
        except KeyError:
            dt_feature_importance[selected_feat_names[idx]]["count"] = 1
            dt_feature_importance[selected_feat_names[idx]]["importances"] = [importances[x]]


print(dtResults)

STARTING Max Depth 10
STARTING Max Depth 50
STARTING Max Depth 100
STARTING Max Depth None
{'MaxDepth10': {'mae': 82.5697016324016, 'mse': 42796.51522047903, 'rmse': 206.87318632553382, 'r2': 0.4932518769410217}, 'MaxDepth50': {'mae': 40.19006682345901, 'mse': 23210.514305829132, 'rmse': 152.3499731074119, 'r2': 0.7251672361845908}, 'MaxDepth100': {'mae': 40.165132561672415, 'mse': 23198.939656900307, 'rmse': 152.31198133075515, 'r2': 0.725304290138389}, 'MaxDepthNone': {'mae': 40.165132561672415, 'mse': 23198.939656900307, 'rmse': 152.31198133075515, 'r2': 0.725304290138389}}


In [ ]:
dtGridSearchResults = pd.DataFrame.from_dict(dtResults).T.reset_index()

dtGridSearchResults['max_depth'] = dtGridSearchResults['index'].astype(int)
dtGridSearchResults.drop('index', inplace=True)
dtGridSearchResults = dtGridSearchResults.melt(id_vars=["max_depth"])
print(dtGridSearchResults)
dtMetricsChart = alt.Chart(dtGridSearchResults).mark_bar().encode(
    x='max_depth:N',
    y='value:Q',
    color=alt.Color('variable:N', scale = alt.Scale(range=["#EAC2B1", "#90C6FA", "#F8E467", "#2C7AAF"])),
    row='max_depth:N',
    column='variable:N',
).resolve_scale(
  y='independent'
).properties(
    title={
    "text": ["Performance Metrics by max_depth"],
      "subtitle": ["Decision Tree Model"]
    }
)

save(dtMetricsChart, '../reports/figures/GridSearchDecisionTree.html')
dtMetricsChart

----